In [ ]:
'''
Задача: выполнить кластеризацию магазинов, описать результаты кластеризации.
'''

In [204]:
import pandas as pd


# Откроем данные
sales_data = pd.read_excel('CL_Data.xlsx', sheet_name='Sales_Data')
store_data = pd.read_excel('CL_Data.xlsx', sheet_name='Store_Data')
print(store_data)

     Store               Status Opening_Date  Square  Assortment_width  \
0        1               Opened   2015-07-03  1047.0            7773.0   
1        2               Opened   2015-06-17  1005.0            8080.0   
2        3               Opened   2007-05-31   932.0            7327.0   
3        4               Opened   2015-06-22   789.0            6508.0   
4        5               Opened   2015-06-01  1067.0            8147.0   
..     ...                  ...          ...     ...               ...   
114    115               Opened   2016-03-23  1092.0            5842.0   
115    116  Will be opened soon   2016-04-04   968.0            6033.0   
116    117               Opened   2016-02-15   900.0            6537.0   
117    118               Opened   2016-03-19   940.0            5552.0   
118    119               Opened   2016-03-24  1160.0            6660.0   

      Latitude  Longitude License  Alco_Forbid  Tobacco_Forbid  \
0    62.089544  42.995538       X          Na

In [208]:
# Даты - 01-02-2016 - 30-04-2016 (3 месяца)
# Магазинов предствлено в сейлс_дата - 103

print(sales_data.head(5))

# + или - справа от фичи означают её полезность.
# - Наиболее продаваемая Lvl2 категория -
# - Наиболее продаваемая Lvl3 категория +
# - Средний (за три месяца) доход на квадрат (в месяц) +
# - Сколько дней работает магазин с открытия +
# - Самая представленная Lvl3 категория (можно будет сравнить с самой продаваемой Lvl3 и выделить еще фичу) -
# - Можно попробовать выделить фичу - процент экономных/транжир среди покупателей +
# - Фича - магазины рядом (например, [2, 4, 7]. Можно использовать при формировании ассортимента близлежащих
# магазинов)

   Store       Date  Quantity_sold  Revenue  Num_of_soc_SKU  Num_of_promo_SKU  \
0      1 2016-02-01             27  4240.21               1                13   
1      1 2016-02-01              3   188.02               1                 0   
2      1 2016-02-01              0     0.00               1                 2   
3      1 2016-02-01             43  7700.36               1                 2   
4      1 2016-02-01              0     0.00               1                 3   

   Lvl1                 Lvl2    Lvl3  Num_of_SKU_in_lvl  
0  ROOT  Алкольная продукция  Вина_1                190  
1  ROOT  Алкольная продукция  Вина_2                  7  
2  ROOT  Алкольная продукция  Вина_3                 22  
3  ROOT  Алкольная продукция   Водка                 80  
4  ROOT  Алкольная продукция  Коньяк                 44  


In [209]:
# Наиболее продаваемая Lvl2 категория (Во всех магазинах это бакалейные товары. Фича не имеет смысла)
lvl2 = sales_data[['Store', 'Lvl2', 'Revenue']].groupby(['Store', 'Lvl2']).sum()
lvl2 = lvl2.sort_values(by=['Store', 'Revenue'])

for i in sales_data['Store'].unique():
    # Вытаскиваем самую продаваемую категорию
    print('Store {0}: {1}'.format(i, lvl2.loc[lvl2['Revenue'] == lvl2.loc[i]['Revenue'].max()].index[0][1]))
    break

Store 1: Бакалейные товары


In [210]:
# Наиболее продаваемая Lvl3 категория (Очень много пива)
lvl3 = sales_data[['Store', 'Lvl3', 'Revenue']].groupby(['Store', 'Lvl3']).sum()
lvl3 = lvl3.sort_values(by=['Store', 'Revenue'])

for i in sales_data['Store'].unique():
    # Вытаскиваем самую продаваемую категорию
    most_lvl3 = lvl3.loc[lvl3['Revenue'] == lvl3.loc[i]['Revenue'].max()].index[0][1]
    
    # Засовываем в стор_дата
    store_data.loc[store_data['Store'] == i, 'MostLvl3'] = most_lvl3
print(store_data['MostLvl3'].unique())

['Пиво' 'Корма_1' 'Табак' 'Кофе' 'Конфеты' 'Сахар' 'Шоколад_1' 'Масло' nan]


In [211]:
# Средний доход на квадрат (Один из важнейших финансовых показателей магазина)
for i in sales_data['Store'].unique():
    # Выделение временного дф для магазина
    tmp_data = sales_data.loc[sales_data['Store'] == i]
    # Подсчет средней суммы за три месяца
    av_r = (tmp_data.loc[tmp_data['Date'] == '2016-02', 'Revenue'].sum() +
            tmp_data.loc[tmp_data['Date'] == '2016-03', 'Revenue'].sum() +
            tmp_data.loc[tmp_data['Date'] == '2016-04', 'Revenue'].sum()) / 3
    # print(av_r)
    
    # Запишем данные в стор_дата
    store_data.loc[store_data['Store'] == i, 'AverageRevenue'] = av_r
# Разделим доход на площадь
store_data['Revenue/S'] = store_data['AverageRevenue'] / store_data['Square']
print(store_data['Revenue/S'].describe())

count    102.000000
mean     448.652429
std      183.031461
min        0.000000
25%      361.823166
50%      460.377933
75%      552.849345
max      935.366774
Name: Revenue/S, dtype: float64


In [212]:
import datetime as dt


# Сколько дней работает магазин с открытия
today = dt.datetime.now()
store_data['DaysWorks'] = (today - store_data['Opening_Date']).dt.days
print(store_data['DaysWorks'])

0      1789.0
1      1805.0
2      4744.0
3      1800.0
4      1821.0
        ...  
114    1525.0
115    1513.0
116    1562.0
117    1529.0
118    1524.0
Name: DaysWorks, Length: 119, dtype: float64


In [213]:
# Самая представленная Lvl3 категория (За исключением 10 магазинов, это везде вина. Не имеет смысла добавлять фичу)
lvl3_represent = sales_data[['Store', 'Lvl3', 'Num_of_SKU_in_lvl']].groupby(['Store', 'Lvl3']).sum()
lvl3_represent = lvl3_represent.sort_values(by=['Store', 'Num_of_SKU_in_lvl'])

wines_count = 0
for i in sales_data['Store'].unique():
    # Вытаскиваем самую представленную категорию
    most_lvl3_represent = lvl3_represent.loc[lvl3_represent['Num_of_SKU_in_lvl'] == lvl3_represent.loc[i]['Num_of_SKU_in_lvl'].max()].index[0][1]
    if most_lvl3_represent == 'Вина_1':
        wines_count += 1
print('Количество магазинов, где Вина_1 имеют наибольшую представленность - {}'.format(wines_count))

Количество магазинов, где Вина_1 имеют наибольшую представленность - 93


In [216]:
'''Время выполнения окна ~30 минут
Класс населения можно охарактеризовать через разницу между средним и медианным значением. В данном случае
довольно сложная ситуация, потому что необходимо анализировать данные по продажам в каждом магазине и категории,
учитывая данные из других магазинов и категорий.
'''
import numpy as np


# Получение общей колонки - средняя цена
sales_data['av_price'] = sales_data['Revenue'].div(sales_data['Quantity_sold'].where(sales_data['Quantity_sold'] != 0, np.nan))

# Группировка по категории и магазину
klas_by_lvl = sales_data[['Lvl3', 'Store', 'av_price']].groupby(['Lvl3', 'Store']).mean()
klas_by_lvl = klas.sort_values(by=['Store'])

# Выделим среднее знач и медиану среди всех магазинов по категории. Сделаем дф на основе данных
category_mean = []
category_med = []
categories = []
qdown_lst = []
qup_lst = []
for i in sales_data['Lvl3'].unique():
    categories.append(i)
    category_mean.append(klas.loc[i].mean().iloc[0])
    category_med.append(klas.loc[i].median().iloc[0])
    qdown_lst.append(klas.loc[i].quantile(0.15).iloc[0])
    qup_lst.append(klas.loc[i].quantile(0.85).iloc[0])
    
cat_df = pd.DataFrame({'Category': categories,
                       'Mean': category_mean,
                       'Median': category_med,
                       'QDOWN': qdown_lst,
                       'QUP': qup_lst})

# Цикл для заполнения данных в sales_data
time1 = dt.datetime.now()
sales_data['Premium'] = [np.nan for i in range(len(sales_data))]
for i in sales_data['Store'].unique():
    for j in sales_data['Lvl3'].unique():
        # Среднее, медиана и квантили категории
        mean = cat_df.loc[cat_df['Category'] == j, 'Mean'].iloc[0]
        median = cat_df.loc[cat_df['Category'] == j, 'Median'].iloc[0]
        q_down = cat_df.loc[cat_df['Category'] == j, 'QDOWN'].iloc[0]
        q_up = cat_df.loc[cat_df['Category'] == j, 'QUP'].iloc[0]
        
        # Реальное значение магазина в категории (try нужен, тк в некоторых магазинах нет некоторых категорий)
        try:
            real_value = sales_data.loc[(sales_data['Store'] == i) &
                                        (sales_data['Lvl3'] == j), 'av_price'].mean()
            # print('real_value - {}'.format(real_value))
        except IndexError as error:
            continue
        
        # Mean > med?
        if mean > median:
            premium = True
        else:
            premium = False
        
        # Теперь, в зависимости от положения real value относительно квантилей, проставим значения
        if premium and (real_value > q_up):
            sales_data.loc[(sales_data['Store'] == i) &
                           (sales_data['Lvl3'] == j), 'Premium'] = 'Premium'
        elif premium and (real_value < q_down):
            sales_data.loc[(sales_data['Store'] == i) &
                           (sales_data['Lvl3'] == j), 'Premium'] = 'Poor'
        elif not premium and (real_value < q_down):
            sales_data.loc[(sales_data['Store'] == i) &
                           (sales_data['Lvl3'] == j), 'Premium'] = 'Poor'
        elif not premium and (real_value > q_up):
            sales_data.loc[(sales_data['Store'] == i) &
                           (sales_data['Lvl3'] == j), 'Premium'] = 'Premium'
        else:
            sales_data.loc[(sales_data['Store'] == i) &
                           (sales_data['Lvl3'] == j), 'Premium'] = 'Medium'

In [222]:
# Из новой колонки можем выделить две фичи - процент "премиум" покупателей и процент "экономных" покупателей
for i in sales_data['Store'].unique():
    # Подсчет процента премиум и экономных покупателей
    val_counts = sales_data.loc[sales_data['Store'] == i, 'Premium'].value_counts(normalize=True)
    
    # Запись данных в стор_дата
    store_data.loc[store_data['Store'] == i, 'Premium%'] = val_counts['Premium']
    store_data.loc[store_data['Store'] == i, 'Economy%'] = val_counts['Poor']
print(store_data['Premium%'].describe())
print(store_data['Economy%'].describe())

count    103.000000
mean       0.154227
std        0.098309
min        0.025974
25%        0.077922
50%        0.129870
75%        0.199981
max        0.478261
Name: Premium%, dtype: float64
count    103.000000
mean       0.152657
std        0.099806
min        0.012993
25%        0.077922
50%        0.129870
75%        0.206460
max        0.467476
Name: Economy%, dtype: float64
